In [ ]:
#basic data compilation

import pandas as pd
import duckdb
import os
import pycountry
import re

# Read IRENA capacity and generation files
df_irena_c = pd.read_excel("data/existing_stock/IRENASTAT-C.xlsx")
df_irena_g = pd.read_excel("data/existing_stock/IRENASTAT-G.xlsx")


df_ember = pd.read_csv("data/existing_stock/yearly_full_release_long_format.csv")

df_gem = pd.read_excel("data/existing_stock/Global-Integrated-Power-April-2025.xlsx", sheet_name="Power facilities")
df_gem_map = pd.read_excel("assumptions/VS_mappings.xlsx", sheet_name="gem_techmap")
df_irena_ember_map = pd.read_excel("assumptions/VS_mappings.xlsx", sheet_name="irena_ember_typemap")

# Remove records where Status starts with 'cancelled', 'shelved', or 'retired'
df_gem = df_gem[~df_gem['Status'].str.lower().str.startswith(('cancelled', 'shelved', 'retired'))]

# Technology = Type for if missing
df_gem['Technology'] = df_gem.apply(
    lambda row: row['Type'] if pd.isna(row['Technology']) else row['Technology'], axis=1
)

# Addin the key field model_fuel to GEM
def custom_fuel(row):
    if row['Type'] != 'oil/gas':
        if row['Type'] == 'hydropower':
            return 'hydro'
        else:
            return row['Type']
    else:
        if pd.notna(row['Fuel']):
            fuel_val = str(row['Fuel'])
        else:
            fuel_val = ''
        if fuel_val.lower().startswith('fossil liquids:'):
            return 'oil'
        else:
            return 'gas'

df_gem['model_fuel'] = df_gem.apply(custom_fuel, axis=1)

# Clean country names in IRENA dataframes
def clean_country_name(country_name):
    """
    Remove trailing information in brackets, e.g., 'Country Name (the)' -> 'Country Name'
    """
    if isinstance(country_name, str):
        # Remove any trailing bracketed info, e.g., "Country (the)" -> "Country"
        return re.sub(r'\s*\(.*\)\s*$', '', country_name).strip()
    return country_name


df_irena_c['Country/area'] = df_irena_c['Country/area'].apply(clean_country_name)
df_irena_g['Country/area'] = df_irena_g['Country/area'].apply(clean_country_name)


df_ember = df_ember.rename(columns={'Variable': 'Type', 'Country code': 'iso_code'})
df_ember = df_ember[df_ember['Subcategory'] == 'Fuel']

#include model_fuel
df_ember = duckdb.sql("SELECT T2.model_fuel,T1.* FROM df_ember T1 inner join df_irena_ember_map T2 ON T1.Type=T2.Type and T2.Source='EMBER'").df()

# Rename 'Technology' column to 'Type' in IRENA dataframes if present
df_irena_c = df_irena_c.rename(columns={'Technology': 'Type'})
df_irena_g = df_irena_g.rename(columns={'Technology': 'Type'})

# add iso_code to df_irena_c and df_irena_g
 # Special arrangement to recognize the following:
    # Kosovo, Chinese Taipei, Republic of Korea, China, Hong Kong Special Administrative Region, Democratic Republic of the Congo
def get_iso_code(country_name):
    if not isinstance(country_name, str):
        return None
    name = country_name.strip().lower()
    # Manual mappings for special cases
    special_cases = {
        'kosovo': 'XKX',
        'kosovo (under unscr 1244/99)': 'XKX',
        'chinese taipei': 'TWN',
        'republic of korea': 'KOR',
        'china, hong kong special administrative region': 'HKG',
        'democratic republic of the congo': 'COD',
        'Russia': 'RUS',
        'DR Congo': 'COD',
        'Kosovo': 'XKX'
    }
    if name in special_cases:
        return special_cases[name]
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except (LookupError, AttributeError, ImportError):
        return None


#Add ISO codes to IRENA and GEM dataframes
df_irena_c['iso_code'] = df_irena_c['Country/area'].apply(get_iso_code)
df_irena_g['iso_code'] = df_irena_g['Country/area'].apply(get_iso_code)
df_gem['iso_code'] = df_gem['Country/area'].apply(get_iso_code)

df_irena_c = duckdb.sql("SELECT T2.model_fuel,T1.* FROM df_irena_c T1 inner join df_irena_ember_map T2 ON T1.Type=T2.Type and T2.Source='IRENA'").df()
df_irena_g = duckdb.sql("SELECT T2.model_fuel,T1.* FROM df_irena_g T1 inner join df_irena_ember_map T2 ON T1.Type=T2.Type and T2.Source='IRENA'").df()

# IAMC data
downscaled_file_path = "data/NGFS4.2/Downscaled_MESSAGEix-GLOBIOM 1.1-M-R12_data.xlsx"
if os.path.exists(downscaled_file_path):
    ngfs_df = pd.read_excel(downscaled_file_path)
else:
    print(f"File not found: {downscaled_file_path}")

# UNSD
"""
sector in ('Power','Industry','Residential','Services','Agriculture','Other','Energy','imports','exports')
attribute in ('transformation','consumption','import','export')  
"""

df_unsd = pd.read_csv("data/unsd/unsd_july_2025.csv")

df_unsd_regmap = pd.read_excel("assumptions/VS_mappings.xlsx", sheet_name="unsd_region_map")
df_unsd_prodmap = pd.read_excel("assumptions/VS_mappings.xlsx", sheet_name="unsd_product_map")
df_unsd_flowmap = pd.read_excel("assumptions/VS_mappings.xlsx", sheet_name="unsd_flow_map")

df_unsd_trade = duckdb.sql(f"""
    SELECT T1.TIME_PERIOD AS year,round(SUM(cast(T1.OBS_VALUE as float) * T1.CONVERSION_FACTOR / 1000 / 3.6),1) as twh_UNSD
    ,T3.attribute,T4.ISO
    from 
    df_unsd T1
    inner join df_unsd_prodmap T2 ON cast(T2.Code as varchar) = cast(T1.COMMODITY as varchar)
    inner join df_unsd_flowmap T3 ON cast(T3.Code as varchar) = cast(T1.TRANSACTION as varchar)
    inner join df_unsd_regmap T4 ON cast(T4.Code as varchar) = cast(T1.REF_AREA as varchar)
    where 
    commodity = 7000 AND
     T3.attribute.lower() IN ('import','export') AND
     T1.TIME_PERIOD >= 2000
    group by T1.TIME_PERIOD,T3.attribute,T4.ISO
""").df()


df_electricity_trade = df_unsd_trade.pivot(index=["ISO", "attribute"], columns="year", values="twh_UNSD")



In [ ]:
# collect information to be able to choose between EMBER and IRENA for utilization factors and efficiency tuning to calibrate to UNSD fuel consumption numbers.
def calibration_data(input_iso,df_irena_util,df_ember_util,df_grouped_gem):

    import duckdb
    import pandas as pd
    import xlwings as xw

    duckdb.register('df_irena_util', df_irena_util)
    duckdb.register('df_ember_util', df_ember_util)
    duckdb.register('df_grouped_gem', df_grouped_gem)


    result = duckdb.sql("""
        SELECT model_fuel,sum(cast(Capacity_GW as float)) as Capacity_GW
        FROM df_grouped_gem
        where "Start year" <= '2022'
        GROUP BY model_fuel
        order by model_fuel
    """).df()

    result = duckdb.sql("""
        SELECT model_fuel,sum(cast(Capacity_GW as float)) as Capacity_GW
        FROM df_ember_util
        where "year" = '2022'
        GROUP BY model_fuel
        order by model_fuel
    """).df()

    result = duckdb.sql("""
        SELECT model_fuel,sum(cast(Capacity_GW as float)) as Capacity_GW
        FROM df_irena_util
        where "year" = '2022'
        GROUP BY model_fuel
        order by model_fuel
    """).df()

    result = duckdb.sql("""
        SELECT T1.iso_code, T1.model_fuel,
        round(SUM(T1.capacity_gw),1) as capacity_gw_gem,
        round(T2.capacity_gw,1) as capacity_gw_irena,
        round(T3.capacity_gw,1) as capacity_gw_ember,
        
        round(SUM(T1.capacity_gw * T2.utilization_factor * 8.76),1) AS generation_twh_gem_irena,
        round(SUM(T1.capacity_gw * T3.utilization_factor * 8.76),1) AS generation_twh_gem_ember,
        round(T2.generation_twh,1) as generation_twh_irena,round(T3.generation_twh,1) as generation_twh_ember,
        
        round(T2.utilization_factor,2) as utilization_factor_irena,
        round(T3.utilization_factor,2) as utilization_factor_ember,
        round(SUM(T1.capacity_gw * coalesce(T1.efficiency, 1)) / SUM(T1.capacity_gw),2) AS avg_efficiency,
        round(SUM(T1.capacity_gw * T2.utilization_factor * 8.76 / coalesce(T1.efficiency, 1)),1) AS fuel_consumed_twh_irena,
        round(SUM(T1.capacity_gw * T3.utilization_factor * 8.76 / coalesce(T1.efficiency, 1)),1) AS fuel_consumed_twh_ember
        
        FROM df_grouped_gem T1
        LEFT join df_irena_util T2 ON T1.model_fuel = T2.model_fuel AND T2.year = '2022'
        LEFT join df_ember_util T3 ON T1.model_fuel = T3.model_fuel AND T3.year = '2022'
        where "Start year" <= '2022'
        GROUP BY T1.iso_code, T1.model_fuel,T2.generation_twh,T3.generation_twh,T2.utilization_factor,T3.utilization_factor,T2.capacity_gw,T3.capacity_gw
        order by T1.iso_code, T1.model_fuel
    """).df()



    df_unsd_iso = duckdb.sql(f"""
        SELECT T1.TIME_PERIOD AS year,T2.model_fuel,round(SUM(cast(T1.OBS_VALUE as float) * T1.CONVERSION_FACTOR / 1000 / 3.6),1) as fuel_consumed_unsd_twh
        from 
        df_unsd T1
        inner join df_unsd_prodmap T2 ON cast(T2.Code as varchar) = cast(T1.COMMODITY as varchar)
        inner join df_unsd_flowmap T3 ON cast(T3.Code as varchar) = cast(T1.TRANSACTION as varchar)
        where 
        -- TIME_PERIOD = 2020 AND
        REF_AREA IN (select code from df_unsd_regmap where ISO = '{input_iso}')
        AND T3.attribute.lower() = 'transformation'
        AND T3.sector.lower() = 'power'
        group by T1.TIME_PERIOD,T2.model_fuel
    """).df()


    # Export the first DataFrame as three separate tables (GW cols, then TWh cols, then remaining cols), one below the other, then the second DataFrame

    # Identify GW columns (ending with '_gw'), TWh columns (ending with '_twh'), and the rest
    gw_cols = result.columns[:2].tolist() + [col for col in result.columns if '_gw' in col]
    twh_cols = result.columns[:2].tolist() + [col for col in result.columns if 'generation_twh' in col]
    other_cols = result.columns[:2].tolist() + [col for col in result.columns if col not in gw_cols + twh_cols]
    base_year_cols = result.columns[:2].tolist() + [col for col in result.columns if 'generation_twh' in col or 'utilization' in col]

    # Prepare the three tables
    df_gw = result[gw_cols]
    df_twh = result[twh_cols]
    df_other = result[other_cols]
    df_base_year = result[base_year_cols]
    
    output_path = f"output/VerveStacks_{input_iso}.xlsx"

    # Write df_unsd_iso to the right of df_other (i.e., as additional columns)
    # First, align df_unsd_iso to have the same number of rows as df_other (if possible)
    # We'll join on 'model_fuel' if present in both, else just concat columns


    df_unsd_iso = df_unsd_iso[df_unsd_iso['year'] == 2022]
    df_unsd_iso = df_unsd_iso.drop(columns=['year'])


    # Check if 'model_fuel' is in both DataFrames to allow merge
    if 'model_fuel' in df_other.columns and 'model_fuel' in df_unsd_iso.columns:
        df_combined = pd.merge(df_other, df_unsd_iso, on='model_fuel', how='left')
    else:
        # fallback: concat as columns (may misalign if row counts differ)
        df_combined = pd.concat([df_other.reset_index(drop=True), df_unsd_iso.reset_index(drop=True)], axis=1)


    # Function to delete a sheet if it exists in the Excel file
    def delete_sheet_if_exists(file_path, sheet_name):
        try:
            app = xw.App(visible=False)
            wb = app.books.open(file_path)
            if sheet_name in [ws.name for ws in wb.sheets]:
                wb.sheets[sheet_name].delete()
                wb.save()
            wb.close()
            app.quit()
        except Exception as e:
            print(f"Warning: Could not delete sheet '{sheet_name}' from {file_path}: {e}")

    # Example usage: delete the 'Calibration' sheet if it exists before writing
    delete_sheet_if_exists(output_path, 'Calibration')


    # Using xlwings to write multiple tables to the same sheet
    app = xw.App(visible=False)
    try:
        wb = app.books.open(output_path)
        
        # Create or get the Calibration sheet
        if 'Calibration' in [ws.name for ws in wb.sheets]:
            ws_calib = wb.sheets['Calibration']
            ws_calib.clear()
        else:
            ws_calib = wb.sheets.add('Calibration')
        
        startrow = 1  # xlwings uses 1-based indexing
        # Write GW columns table
        ws_calib.range(f'A{startrow}').value = [df_gw.columns.tolist()] + df_gw.values.tolist()
        startrow += len(df_gw) + 3  # leave a blank row

        # Write TWh columns table
        ws_calib.range(f'A{startrow}').value = [df_twh.columns.tolist()] + df_twh.values.tolist()
        startrow += len(df_twh) + 3  # leave a blank row

        # Write other columns table
        ws_calib.range(f'A{startrow}').value = [df_combined.columns.tolist()] + df_combined.values.tolist()
        
        # Create or get the base_year_data sheet
        if 'base_year_data' in [ws.name for ws in wb.sheets]:
            ws_base = wb.sheets['base_year_data']
            ws_base.clear()
        else:
            ws_base = wb.sheets.add('base_year_data')
        
        # Write base year data
        ws_base.range('A1').value = [df_base_year.columns.tolist()] + df_base_year.values.tolist()
        
        wb.save()
        wb.close()
    finally:
        app.quit()


In [ ]:

def get_iamc_data(input_iso):


    # collect data to construct future scenarios using published IAMC results
    import matplotlib.pyplot as plt
    import math

    # Read the Downscaled_MESSAGEix-GLOBIOM 1.1-M-R12_data file and filter on Region='IND'
    df_varbl = pd.read_excel("assumptions/VS_mappings.xlsx", sheet_name="iamc_variables")
    # Filter for potential_use in 'demand_projection' or 'fuel_supply'
    df_varbl = df_varbl[df_varbl['potential_use'].isin(['demand_projection', 'fuel_supply'])]

    sheet_name = 'iamc_data'

    ngfs_df_iso = ngfs_df[ngfs_df['Region'] == input_iso]
    ngfs_df_iso = ngfs_df_iso[ngfs_df_iso['Variable'].isin(df_varbl['variable'])]
    ngfs_df_iso = ngfs_df_iso.dropna(axis=1, how='all')


    # display(ngfs_df_iso)

    # For each variable, plot a line chart with scenario in the legend, sorted by the 'potential_use' column,
    # and display the unit as the y-axis label. Display the charts in two columns.


    # Merge in the 'potential_use' and 'unit' info for sorting and labeling
    df_varbl_for_merge = df_varbl[['variable', 'potential_use','commodity']].drop_duplicates()
    ngfs_df_iso_with_pu = ngfs_df_iso.merge(
        df_varbl_for_merge,
        left_on='Variable',
        right_on='variable',
        how='left'
    )

    # Get unique variables sorted by 'potential_use'
    variables_sorted = (
        ngfs_df_iso_with_pu[['Variable', 'potential_use', 'Unit']]
        .drop_duplicates()
        .sort_values(by='potential_use', na_position='last')
        .reset_index(drop=True)
    )

    years = [col for col in ngfs_df_iso.columns if str(col).isdigit()]

    n_vars = len(variables_sorted)
    n_cols = 2
    n_rows = math.ceil(n_vars / n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5 * n_rows), squeeze=False)
    axes = axes.flatten()

    for idx, row in variables_sorted.iterrows():
        var = row['Variable']
        unit = row['Unit'] if pd.notnull(row['Unit']) else "Value"
        ax = axes[idx]
        df_plot = ngfs_df_iso[ngfs_df_iso['Variable'] == var]
        if df_plot.empty:
            ax.set_visible(False)
            continue
        for scenario, group in df_plot.groupby('Scenario'):
            yvals = group[years].values
            if yvals.shape[0] > 1:
                yvals = yvals.mean(axis=0)
            else:
                yvals = yvals.flatten()
            ax.plot(years, yvals, marker='o', label=scenario)
        ax.set_title(f"{var} ({input_iso})")
        ax.set_xlabel("Year")
        ax.set_ylabel(unit)
        ax.legend(title="Scenario")
        ax.grid(True, linestyle='--', alpha=0.5)

    # Hide any unused subplots
    for j in range(idx + 1, len(axes)):
        axes[j].set_visible(False)

    plt.tight_layout()
    # plt.show()

    import io

    output_path = f"output/VerveStacks_{input_iso}.xlsx"


    # Save the figure to a BytesIO buffer
    imgdata = io.BytesIO()
    fig.savefig(imgdata, format='png', bbox_inches='tight')
    imgdata.seek(0)

    # Using xlwings to add image to Excel file
    app = xw.App(visible=False)
    try:
        if os.path.exists(output_path):
            wb = app.books.open(output_path)
        else:
            # If file doesn't exist yet, create a new workbook
            wb = app.books.add()
            # Remove the default sheet if present
            if len(wb.sheets) > 0:
                for sheet in wb.sheets:
                    if sheet.name.startswith('Sheet'):
                        sheet.delete()

        # Remove "iamc_charts" if it already exists
        if "iamc_charts" in [ws.name for ws in wb.sheets]:
            wb.sheets["iamc_charts"].delete()

        ws = wb.sheets.add("iamc_charts")

        # Save the image to a temporary file first, then insert
        import tempfile
        with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
            fig.savefig(temp_file.name, format='png', bbox_inches='tight')
            temp_file_path = temp_file.name
        
        # Insert the image
        ws.pictures.add(temp_file_path, left=ws.range('A1').left, top=ws.range('A1').top)
        
        # Clean up temp file
        os.unlink(temp_file_path)

        # Save the workbook
        wb.save(output_path)
        wb.close()
    finally:
        app.quit()


    if os.path.exists(output_path):
        # Using xlwings to write data to Excel
        app = xw.App(visible=False)
        try:
            wb = app.books.open(output_path)
            
            # Create or replace the sheet
            if sheet_name in [ws.name for ws in wb.sheets]:
                wb.sheets[sheet_name].delete()
            
            ws = wb.sheets.add(sheet_name)
            
            # Write the data
            df_to_write = ngfs_df_iso_with_pu.sort_values(by='potential_use')
            ws.range('A1').value = [df_to_write.columns.tolist()] + df_to_write.values.tolist()
            
            wb.save()
            wb.close()
        finally:
            app.quit()


In [ ]:
# compile new tech characteristics from WEO data - for the appropriate region for alternative scenarios.
def get_weo_data(input_iso):

    import pandas as pd
    from pandas.core.window.numba_ import generate_manual_numpy_nan_agg_with_axis


    # Define paths for the uploaded files
    weo_path = "data/technologies/WEO_2024_PG_Assumptions_STEPSandNZE_Scenario.xlsb"
    tech_mapping_path = "data/technologies/ep_technoeconomic_assumptions.xlsx"
    output_path = f"output/VerveStacks_{input_iso}.xlsx"

    # Load region mapping
    reg_map_df = pd.read_excel(tech_mapping_path, sheet_name="ep_regionmap")
    region_name = reg_map_df.loc[reg_map_df['iso'] == input_iso, 'region'].values[0]
    

    # Define relevant WEO tech assumption sheets
    relevant_sheets = [
        "Renewables",
        "Fossil fuels equipped with CCUS",
        "Nuclear",
        "Gas",
        "Coal"
    ]


    # Function to process and clean each sheet
    def process_weo_sheet(sheet_name, region_name):
        df = pd.read_excel(weo_path, sheet_name=sheet_name, engine="pyxlsb")

        # Create consistent column headers
        orig_cols = list(df.columns)
        new_header_row = []
        last_name = None
        for col in orig_cols:
            if not (str(col).startswith("Unnamed") or pd.isna(col)):
                last_name = col
            new_header_row.append(last_name)
        df1 = pd.concat([pd.DataFrame([new_header_row], columns=df.columns), df], ignore_index=True)

        # Add attribute row
        second_row = df1.iloc[1]
        filled_second_row = []
        last_val = None
        for val in second_row:
            if pd.notna(val):
                last_val = val
            filled_second_row.append(last_val)
        df2 = pd.concat([df1.iloc[[0]], pd.DataFrame([filled_second_row], columns=df.columns), df1.iloc[1:]], ignore_index=True)

        # Add Technology column
        tech_col = []
        current_tech = None
        for idx, row in df2.iterrows():
            if pd.isna(row.iloc[2]) or str(row.iloc[2]).strip() == '':
                if pd.notna(row.iloc[0]) and str(row.iloc[0]).strip() != '':
                    current_tech = row.iloc[0]
            tech_col.append(current_tech)
        df2.insert(0, "Technology", tech_col)

        # Identify row where years start
        year_row_idx = None
        for idx, val in enumerate(df2.iloc[:, 2]):
            if pd.notna(val) and str(val).isdigit() and len(str(val)) == 4:
                year_row_idx = idx
                break
        if year_row_idx is None:
            year_row_idx = 1

        # Filter by region name
        df_out = pd.concat([
            df2.iloc[:year_row_idx+1],
            df2.iloc[year_row_idx+1:][df2.iloc[year_row_idx+1:, 1] == region_name]
        ], ignore_index=True)
        return df_out


    flat_rows = []


    for sheet in relevant_sheets:
        df_sheet = process_weo_sheet(sheet, region_name)

        # Find the year row index for this sheet
        year_row_idx = None
        for idx, val in enumerate(df_sheet.iloc[:, 2]):
            if pd.notna(val) and str(val).isdigit() and len(str(val)) == 4:
                year_row_idx = idx
                break
        if year_row_idx is None:
            year_row_idx = 1

        for idx in range(year_row_idx + 1, len(df_sheet)):
            row = df_sheet.iloc[idx]
            technology = row['Technology']
            for col_idx in range(2, len(df_sheet.columns)):
                scenario = df_sheet.iloc[0, col_idx]
                attribute = df_sheet.iloc[1, col_idx]
                year = df_sheet.iloc[year_row_idx, col_idx] if year_row_idx < len(df_sheet) else None
                value = row.iloc[col_idx]
                if pd.notna(year) and pd.notna(value):
                    flat_rows.append({
                        'scenario': scenario,
                        'technology': technology,
                        'attribute': attribute,
                        'year': year,
                        'value': value
                    })

    flat_df = pd.DataFrame(flat_rows)


    # There is no option in pivot_table to ignore non-numeric values directly,
    # so we must filter to numeric values before pivoting.
    flat_df_numeric = flat_df.copy()
    flat_df_numeric['value'] = pd.to_numeric(flat_df_numeric['value'], errors='coerce')
    flat_df_numeric = flat_df_numeric[flat_df_numeric['value'].notna()]
    weo_pg_final = flat_df_numeric.pivot_table(
        index=['scenario', 'technology', 'attribute'],
        columns='year',
        values='value',
        fill_value=''
    ).reset_index()


    # Attempt to guess model_fuel for each technology in pivot_df by matching with df_gem_map
    # Use approximate matching: if an exact match is not found, look for key model_fuel words in the technology string

    # Build a mapping from model_fuel keywords to model_fuel from df_gem_map
    model_fuel_keywords = (
        df_gem_map[['model_fuel']]
        .drop_duplicates()
        .model_fuel
        .dropna()
        .unique()
    )

    # Lowercase set of model_fuel keywords for matching
    model_fuel_keywords = [str(fuel).lower() for fuel in model_fuel_keywords if isinstance(fuel, str)]

    # Create a lowercase mapping from Technology to model_fuel from df_gem_map
    tech_to_fuel = (
        df_gem_map.drop_duplicates(subset=['Technology', 'model_fuel'])
        .set_index(df_gem_map['Technology'].str.lower())['model_fuel']
        .to_dict()
    )
    # Add special cases for mapping technology names to model_fuel
    special_cases = {
        "ccgt": "gas",
        "ccgt + ccs": "gas",
        "ccgt - chp": "gas",
        "fuel cell (distributed electricity generation)": "hydrogen",
        "igcc + ccs": "coal",
        "marine": "hydro",
        "Gas Turbine": "gas",
        "oxyfuel + ccs": "coal"
    }

    # Update tech_to_fuel with special cases (overriding if necessary)
    for tech_name, model_fuel in special_cases.items():
        tech_to_fuel[tech_name.lower()] = model_fuel


    def guess_model_fuel(tech):
        tech_l = str(tech).lower()
        # 1. Try exact match
        if tech_l in tech_to_fuel:
            return tech_to_fuel[tech_l]
        # 2. Try keyword match
        for fuel in model_fuel_keywords:
            if fuel in tech_l:
                return fuel
        # 3. Not found
        return 'unknown'


    # Map from attribute keywords to model_attribute values
    attribute_to_model_attribute = {
        "O&M": "ncap_fom",
        "capacity factor": "ncap_af",
        "capital": "ncap_cost",
        "construction time": "ncap_iled",
        "efficiency": "efficiency"
    }

    def get_model_attribute(attr):
        if not isinstance(attr, str):
            return None
        attr_l = attr.lower()
        for key, value in attribute_to_model_attribute.items():
            if key.lower() in attr_l:
                return value
        return None

    weo_pg_final['model_attribute'] = weo_pg_final['attribute'].apply(get_model_attribute)


    weo_pg_final['model_fuel'] = weo_pg_final['technology'].apply(guess_model_fuel)

    output_path = f"output/VerveStacks_{input_iso}.xlsx"

    if os.path.exists(output_path):
        # Using xlwings to write WEO data to Excel
        app = xw.App(visible=False)
        try:
            wb = app.books.open(output_path)
            
            # Create or replace the sheet
            if 'weo_pg' in [ws.name for ws in wb.sheets]:
                wb.sheets['weo_pg'].delete()
            
            ws = wb.sheets.add('weo_pg')
            
            # Write the data
            ws.range('A1').value = [weo_pg_final.columns.tolist()] + weo_pg_final.values.tolist()
            
            wb.save()
            wb.close()
        finally:
            app.quit()


In [ ]:
# cleaning up GEM data here. Creating the map table for key columns model_fuel and model_tech that will drive all aggregation and calibration
# This is the first step in the process of creating the map table. Not to be used in the regular flow.


agg_capacity = df_gem.groupby(['Type', 'Technology', 'model_fuel', 'Status'], dropna=False)['Capacity (MW)'].sum().reset_index()


status_pivot = agg_capacity.pivot_table(
    index=['Type', 'Technology', 'model_fuel'],
    columns='Status',
    values='Capacity (MW)',
    fill_value=0
).reset_index()

display(status_pivot)



# Using xlwings to write status pivot data
app = xw.App(visible=False)
try:
    wb = app.books.add()
    ws = wb.sheets[0]  # Use the default first sheet
    ws.name = 'status_pivot'
    
    # Write the data
    ws.range('A1').value = [status_pivot.columns.tolist()] + status_pivot.values.tolist()
    
    wb.save('output/VerveStacks_status_pivot.xlsx')
    wb.close()
finally:
    app.quit()


In [ ]:
def ccs_retrofits(input_iso,df_grouped_gem):

    epa_ccs_rf_df = pd.read_excel("data/existing_stock/epa_coal+gas ccs retrofit data.xlsx", sheet_name="epa_ccs_rf")

    duckdb.register('epa_ccs_rf_df', epa_ccs_rf_df)
    duckdb.register('df_grouped_gem', df_grouped_gem)

    # aggregated plants
    result = duckdb.sql("""
        SELECT T1.model_fuel,T1.model_name || '_ccs-rf' as model_name
        ,T2.capex,T2.fixom,T2.varom
        ,(100-T2.heatrate_penalty) * max(T1.efficiency) / 100 AS efficiency
        ,(100-T2.capacity_penalty) * .95 / 100 AS AF
        ,max(T1.efficiency) as efficiency_old
        ,T1.model_name AS plant_old
        FROM df_grouped_gem T1 
        inner join epa_ccs_rf_df T2 ON 
        T1.model_fuel=T2.model_fuel and
        T2.eff1 = 0
        and
        T2.cap1=0

        where T1.model_description ilike 'aggregated%' and T1.model_fuel='coal'

        group by T1.model_fuel,T1.model_name,T2.capex,T2.fixom,T2.varom,T2.heatrate_penalty,T2.capacity_penalty,T1.model_name

        UNION
        
        SELECT T1.model_fuel,T1.model_name || '_ccs-rf' as model_name
        ,T2.capex,T2.fixom,T2.varom
        ,(100-T2.heatrate_penalty) * T1.efficiency / 100 AS efficiency
        ,(100-T2.capacity_penalty) * .95 / 100 AS AF
        ,T1.efficiency as efficiency_old
        ,T1.model_name AS plant_old
        FROM df_grouped_gem T1 
        inner join epa_ccs_rf_df T2 ON 
        T1.model_fuel=T2.model_fuel and
        T1.efficiency < T2.efficiency and T1.efficiency >= T2.eff1
        and
        T1.capacity_gw < T2.capacity/1000 and T1.capacity_gw >= T2.cap1/1000

        where not T1.model_description ilike 'aggregated%' and T1.model_fuel='coal'

        UNION

        SELECT T1.model_fuel,T1.model_name || '_ccs-rf' as model_name
        ,T2.capex,T2.fixom,T2.varom
        ,(100-T2.heatrate_penalty) * max(T1.efficiency) / 100 AS efficiency
        ,(100-T2.capacity_penalty) * .95 / 100 AS AF
        ,max(T1.efficiency) as efficiency_old
        ,T1.model_name AS plant_old
        FROM df_grouped_gem T1 
        inner join epa_ccs_rf_df T2 ON 
        T2.model_fuel='gas'
        
        where T1.model_fuel IN ('gas','oil')

        group by T1.model_fuel,T1.model_name,T2.capex,T2.fixom,T2.varom,T2.heatrate_penalty,T2.capacity_penalty,T1.model_name

        
        """
        ).df()

    output_path = f"output/VerveStacks_{input_iso}.xlsx"
    
    if os.path.exists(output_path):
        # Using xlwings to write CCS retrofits data
        app = xw.App(visible=False)
        try:
            wb = app.books.open(output_path)
            
            # Create or replace the sheet
            if 'ccs_retrofits' in [ws.name for ws in wb.sheets]:
                wb.sheets['ccs_retrofits'].delete()
            
            ws = wb.sheets.add('ccs_retrofits')
            
            # Write the data
            result_sorted = result.sort_values(by='model_name')
            ws.range('A1').value = [result_sorted.columns.tolist()] + result_sorted.values.tolist()
            
            wb.save()
            wb.close()
        finally:
            app.quit()



In [ ]:
def re_targets_ember(input_iso):


    ember_raw_data_long = pd.read_excel("data/ember_targets_download2025jul.xlsx", sheet_name="raw_data_long")
    ember_sources = pd.read_excel("data/ember_targets_download2025jul.xlsx", sheet_name="sources")

    duckdb.register('ember_raw_data_long', ember_raw_data_long)
    duckdb.register('ember_sources', ember_sources)

    result = duckdb.sql(f"""

    select T1.TARGET_YEAR,T1.FUEL_CATEGORY,T1.METRIC,T1.VALUE,
    T2.SOURCE_TYPE,T2.SOURCE_NAME,T2.PUBLISHER,T2.ANNOUNCEMENT_DATE,T2.LINK,T2.SOURCE_SUMMARY,T2.NOTES
    from ember_raw_data_long T1
    inner join ember_sources T2 on T1.SOURCE_ID=T2.SOURCE_ID

    where T1.COUNTRY_CODE='{input_iso}'
    order by T2.SOURCE_TYPE,T1.METRIC,T1.FUEL_CATEGORY

        """
        ).df()


    output_path = f"output/VerveStacks_{input_iso}.xlsx"
        
    if os.path.exists(output_path):
        # Using xlwings to write RE targets data
        app = xw.App(visible=False)
        try:
            wb = app.books.open(output_path)
            
            # Create or replace the sheet
            if 're_targets' in [ws.name for ws in wb.sheets]:
                wb.sheets['re_targets'].delete()
            
            ws = wb.sheets.add('re_targets')
            
            # Write the data
            ws.range('A1').value = [result.columns.tolist()] + result.values.tolist()
            
            wb.save()
            wb.close()
        finally:
            app.quit()



In [ ]:
# create Veda model folder

input_iso='NGA'

from atexit import register
import shutil
import os
import pandas as pd
import duckdb
import openpyxl

def copy_vs_iso_template(input_iso):
    src_folder = "assumptions/VerveStacks_ISO_template"
    dest_folder = f"output/VerveStacks_{input_iso}"

    # If destination folder exists, delete it
    if os.path.exists(dest_folder):
        shutil.rmtree(dest_folder)
    # Copy the template folder to the destination
    shutil.copytree(src_folder, dest_folder)

copy_vs_iso_template(input_iso)
dest_folder = f"output/VerveStacks_{input_iso}"

syssettings_path = os.path.join(dest_folder, "SysSettings.xlsx")
wb_sys = openpyxl.load_workbook(syssettings_path)
ws_sys = wb_sys["system_settings"]
ws_sys["B3"] = input_iso
wb_sys.save(syssettings_path)

existing_stock_df = pd.read_excel(f"output/vervestacks_{input_iso}.xlsx", sheet_name="existing_stock")
ccs_retrofits_df = pd.read_excel(f"output/vervestacks_{input_iso}.xlsx", sheet_name="ccs_retrofits")
weo_pg_df = pd.read_excel(f"output/vervestacks_{input_iso}.xlsx", sheet_name="weo_pg")
iamc_data_df = pd.read_excel(f"output/vervestacks_{input_iso}.xlsx", sheet_name="iamc_data")
base_year_data_df = pd.read_excel(f"output/vervestacks_{input_iso}.xlsx", sheet_name="base_year_data")
life_df = pd.read_excel(f"data/technologies/ep_technoeconomic_assumptions.xlsx", sheet_name="life")

duckdb.register('existing_stock_df', existing_stock_df)
duckdb.register('ccs_retrofits_df', ccs_retrofits_df)
duckdb.register('weo_pg_df', weo_pg_df)
duckdb.register('life_df', life_df)

fi_t = duckdb.sql(f"""
select T1.model_name AS process,T1.model_fuel AS "comm-in",
CASE
    WHEN
    T1.model_fuel = 'solar'
    THEN 'ELC_Sol-' || '{input_iso}'
    WHEN
    T1.model_fuel = 'wind'
    THEN 'ELC_Win-' || '{input_iso}'
    ELSE 'ELC'
END AS "comm-out",
T1."Start year" AS year,T1.capacity_gw AS ncap_pasti,
T1.efficiency AS efficiency,T1.capex AS ncap_cost,T1.fixom AS ncap_fom,T1.varom AS act_cost,
case
    when
    coalesce(T1.retirement_year,2055) - T1."Start year" > T2.life
    then
    coalesce(T1.retirement_year,2055) - T1."Start year" 
    else
    T2.life
end AS ncap_tlife
from existing_stock_df T1
left join life_df T2 on T1.model_name ilike T2.model_name || '%'
order by T1.model_name,T1."Start year"
""").df()

fi_t.loc[fi_t['process'].str.startswith('ep_hydro_ps'), 'comm-in'] = 'ELC'


fi_p = duckdb.sql("""
select 
'ele' AS set,
T1.model_name AS process,T1.model_description AS description,
'GW' AS capacity_unit,
'TWh' AS activity_unit,
CASE when model_fuel IN ('solar','windon','windoff') then 'annual' else 'daynite' end AS timeslicelevel,
CASE when model_description ilike 'aggregated%' then 'yes' else 'no' end AS vintage

from existing_stock_df T1
group by T1.model_name,T1.model_description,T1.model_fuel
order by T1.model_name
""").df()

fi_p.loc[fi_p['process'].str.startswith('ep_hydro_ps'), 'set'] = 'STG'

duckdb.register('fi_p', fi_p)

fi_t_ccs = duckdb.sql("""
select T1.model_name AS process,T1.model_fuel AS "comm-in",'ELC' as "comm-out",
T1.efficiency AS efficiency,T1.capex AS ncap_cost,T1.fixom AS ncap_fom,T1.varom AS act_cost,
T1.AF AS AF,
T1.plant_old AS other_indexes, 1 AS prc_refit,
20 AS ncap_tlife
from ccs_retrofits_df T1

order by T1.model_name
""").df()

fi_p_ccs = duckdb.sql("""
select 
'ele' AS set,
T1.model_name AS process,'ccs retrofit of -- ' || T2.description AS description,
'GW' AS capacity_unit,
'TWh' AS activity_unit,
'daynite' AS timeslicelevel,
'no' AS vintage

from ccs_retrofits_df T1
inner join fi_p T2 ON T1.plant_old = T2.process
group by T1.model_name,T2.description
order by T1.model_name
""").df()

fi_t_weo = duckdb.sql("""
select T1.technology AS process,T1.model_fuel AS "comm-in",'ELC' as "comm-out",
"2023","2030","2050",
T1.model_attribute AS attribute,
from weo_pg_df T1
where T1.scenario ilike 'Stated%'
order by T1.technology,T1.model_attribute
""").df()

fi_p_weo = duckdb.sql("""
select 
'ele' AS set,
T1.technology AS process,'' AS description,
'GW' AS capacity_unit,
'TWh' AS activity_unit,
'daynite' AS timeslicelevel,
'yes' AS vintage

from weo_pg_df T1
group by T1.technology
order by T1.technology
""").df()


# Delete the file if already present, then create vt_vervestacks_{iso}.xlsx in the destination folder
vt_output_path = os.path.join(dest_folder, f"vt_vervestacks_{input_iso}_v1.xlsx")
if os.path.exists(vt_output_path):
    os.remove(vt_output_path)

# Using xlwings to write complex multi-table Excel file
app = xw.App(visible=False)
try:
    wb = app.books.add()
    
    # Create existing_stock sheet
    if wb.sheets[0].name.startswith('Sheet'):
        wb.sheets[0].name = 'existing_stock'
    ws_existing = wb.sheets[0]
    
    # Write fi_t data starting at row 3 (1-based)
    ws_existing.range('A3').value = [fi_t.columns.tolist()] + fi_t.values.tolist()
    
    # Write fi_p data starting at same row but different column
    start_col = fi_t.shape[1] + 3  # +2 for gap, +1 for next column
    ws_existing.range(f'{chr(64 + start_col)}3').value = [fi_p.columns.tolist()] + fi_p.values.tolist()
    
    # Add labels
    ws_existing.range('A2').value = "~fi_t"
    ws_existing.range(f'{chr(64 + start_col)}2').value = "~fi_process"
    
    # Create ccs_retrofits sheet
    ws_ccs = wb.sheets.add('ccs_retrofits')
    
    # Write fi_t_ccs data starting at row 3
    ws_ccs.range('A3').value = [fi_t_ccs.columns.tolist()] + fi_t_ccs.values.tolist()
    
    # Write fi_p_ccs data starting at same row but different column
    start_col_ccs = fi_t_ccs.shape[1] + 3
    ws_ccs.range(f'{chr(64 + start_col_ccs)}3').value = [fi_p_ccs.columns.tolist()] + fi_p_ccs.values.tolist()
    
    # Add labels
    ws_ccs.range('A2').value = "~fi_t"
    ws_ccs.range(f'{chr(64 + start_col_ccs)}2').value = "~fi_process"
    
    # Create weo_pg sheet
    ws_weo = wb.sheets.add('weo_pg')
    
    # Write fi_t_weo data starting at row 3
    ws_weo.range('A3').value = [fi_t_weo.columns.tolist()] + fi_t_weo.values.tolist()
    
    # Write fi_p_weo data starting at same row but different column
    start_col_weo = fi_t_weo.shape[1] + 3
    ws_weo.range(f'{chr(64 + start_col_weo)}3').value = [fi_p_weo.columns.tolist()] + fi_p_weo.values.tolist()
    
    # Add labels
    ws_weo.range('A2').value = "~fi_t"
    ws_weo.range(f'{chr(64 + start_col_weo)}2').value = "~fi_process"
    
    wb.save(vt_output_path)
    wb.close()
finally:
    app.quit()
    

vt_output_path = os.path.join(dest_folder, f"SuppXLS/Scen_Par-NGFS.xlsx")


# Using xlwings to add sheets to existing workbook
if os.path.exists(vt_output_path):
    app = xw.App(visible=False)
    try:
        wb = app.books.open(vt_output_path)
        
        # Add or replace iamc_data sheet
        if "iamc_data" in [ws.name for ws in wb.sheets]:
            wb.sheets["iamc_data"].clear()
        wb.sheets["iamc_data"].range('A1').value = [iamc_data_df.columns.tolist()] + iamc_data_df.values.tolist()
        
        # Add or replace base_year_data sheet
        if "base_year_data" in [ws.name for ws in wb.sheets]:
            wb.sheets["base_year_data"].clear()
        wb.sheets["base_year_data"].range('A1').value = [base_year_data_df.columns.tolist()] + base_year_data_df.values.tolist()
        
        wb.save()
        wb.close()
    finally:
        app.quit()

bvs_output_path = os.path.join(dest_folder, f"SuppXLS/Scen_Base_VS.xlsx")

# Using xlwings to write to second Excel file
if os.path.exists(bvs_output_path):
    app = xw.App(visible=False)
    try:
        wb = app.books.open(bvs_output_path)
        
        # Add or replace base_year_data sheet
        if "base_year_data" in [ws.name for ws in wb.sheets]:
            wb.sheets["base_year_data"].clear()
        wb.sheets["base_year_data"].range('A1').value = [base_year_data_df.columns.tolist()] + base_year_data_df.values.tolist()
        
        wb.save()
        wb.close()
    finally:
        app.quit()


# clean up the RE resource SubRES file
def remove_other_iso(input_iso,file_name):

    RE_SubRES_file = f"output/VerveStacks_{input_iso}/" + file_name
    
    import xlwings as xw

    app = xw.App(visible=False)
    try:
        wb = app.books.open(RE_SubRES_file)

        # Fix: skip rows where cell is None or not a 3-char string
        for ws in wb.sheets:

            cell_a5 = ws.range('A5').value
            if cell_a5 is None or (isinstance(cell_a5, float) and pd.isna(cell_a5)):
                continue  # go to next sheet if A5 is null

            row = 5
            max_row = ws.used_range.last_cell.row if ws.used_range else 5
            while row <= max_row:
                cell = ws.range(f'A{row}').value
                # Only process if cell is a 3-char string
                if isinstance(cell, str) and len(cell) == 3:
                    if cell != input_iso:
                        ws.range(f'{row}:{row}').delete()
                        max_row -= 1  # since a row was deleted
                        continue
                row += 1

        wb.save()
        wb.close()
    finally:
        app.quit()

print("Removing other ISOs from SubRES_REZoning_Sol_Win_andHydro.xlsx")
remove_other_iso(input_iso,'SubRES_Tmpl/SubRES_REZoning_Sol_Win_andHydro.xlsx')


def replace_kinesys_name(input_iso,file_name,sheet_name,kinesys_name):

    scen_file = f"output/VerveStacks_{input_iso}/" + file_name

    # Using xlwings to replace kinesys names
    app = xw.App(visible=False)
    try:
        wb = app.books.open(scen_file)
        ws = wb.sheets[sheet_name]
        
        ws.range('A1').value = input_iso        # to clean up other ISOs

        # Iterate over all columns in row 4 (xlwings uses 1-based indexing)
        if ws.used_range:
            max_col = ws.used_range.last_cell.column
            for col in range(1, max_col + 1):
                cell_value = ws.range((4, col)).value
                if cell_value and str(cell_value).lower() == kinesys_name.lower():
                    ws.range((4, col)).value = input_iso

        # Save the workbook
        wb.save()
        wb.close()
    finally:
        app.quit()

df_kinesys_region_map = pd.read_excel("assumptions/VS_mappings.xlsx", sheet_name="kinesys_region_map")
df_kinesys_region_map = df_kinesys_region_map[df_kinesys_region_map['iso']==input_iso]

print("Replacing kinesys name in Scen_TSParameters_12.xlsx")
replace_kinesys_name(input_iso,'SuppXLS/Scen_TSParameters_12.xlsx','PKFLX',df_kinesys_region_map['kinesys'].iloc[0])
replace_kinesys_name(input_iso,'SuppXLS/Scen_TSParameters_12.xlsx','YRFRs',df_kinesys_region_map['kinesys'].iloc[0])

print("Replacing kinesys name in Scen_TSParameters_108.xlsx")
replace_kinesys_name(input_iso,'SuppXLS/Scen_TSParameters_108.xlsx','PKFLX',df_kinesys_region_map['kinesys'].iloc[0])
replace_kinesys_name(input_iso,'SuppXLS/Scen_TSParameters_108.xlsx','YRFRs',df_kinesys_region_map['kinesys'].iloc[0])

print("Replacing kinesys name in Scen_TSParameters_348.xlsx")
replace_kinesys_name(input_iso,'SuppXLS/Scen_TSParameters_348.xlsx','PKFLX',df_kinesys_region_map['kinesys'].iloc[0])
replace_kinesys_name(input_iso,'SuppXLS/Scen_TSParameters_348.xlsx','YRFRs',df_kinesys_region_map['kinesys'].iloc[0])

import xlwings as xw

# # List of Excel files written to in this block
# excel_files_to_update = [
#     'SubRES_Tmpl/SubRES_REZoning_Sol_Win_andHydro.xlsx',
#     f"output/VerveStacks_{input_iso}/SuppXLS/Scen_TSParameters_12.xlsx",
#     f"output/VerveStacks_{input_iso}/SuppXLS/Scen_TSParameters_108.xlsx",
#     f"output/VerveStacks_{input_iso}/SuppXLS/Scen_TSParameters_348.xlsx",
#     syssettings_path,
#     bvs_output_path
# ]

# # Remove duplicates and ensure files exist before opening
# unique_excel_files = set()
# for file in excel_files_to_update:
#     if os.path.exists(file):
#         unique_excel_files.add(os.path.abspath(file))

# for file_path in unique_excel_files:
#     app = xw.App(visible=False)
#     try:
#         wb = app.books.open(file_path)
#         wb.app.calculate()  # Calculate all formulas in the workbook
#         wb.save()
#         wb.close()
#     finally:
#         app.quit()


print("Done")

In [ ]:
# detailed characteristics of existing stock for a given country

import pandas as pd
import numpy as np
import os

import shutil
import xlwings as xw

import duckdb


input_iso='CHE'
capacity_threshold=300

"""
For a given ISO code, compute utilization factors from IRENA and EMBER,
and export both tables to a single Excel file (two sheets, one for each source).
"""


template_path = "assumptions/vervestacks_ISO_template.xlsx"
output_path = f"output/vervestacks_{input_iso}.xlsx"

# Define a helper function to read a value from a given cell using xlwings
def read_cell(ws, cell):
    value = ws.range(cell).value
    return value



if os.path.exists(output_path):
    app = xw.App(visible=False)
    try:
        wb = app.books.open(output_path)
        ws = wb.sheets['system_settings']
        efficiency_adjustment_gas = ws.range('A4').value
        efficiency_adjustment_coal = ws.range('A5').value
        capacity_threshold = ws.range('A3').value
        wb.close()
    finally:
        app.quit()
else:

    # If the output file already exists and is open in Excel, shutil.copyfile may fail.
    # To handle this, try-except and if it fails, create a new workbook from template.
    try:
        shutil.copyfile(template_path, output_path)
        # Open the newly created Excel file using xlwings
        app = xw.App(visible=False)
        try:
            wb = app.books.open(output_path)

            # Select the 'system_settings' sheet
            if 'system_settings' in [ws.name for ws in wb.sheets]:
                ws = wb.sheets['system_settings']
            else:
                ws = wb.sheets[0]  # fallback if sheet not found

            # Write input_iso to cell A1
            ws.range('A1').value = input_iso
            ws.range('A3').value = capacity_threshold

            # Read variables from the sheet
            efficiency_adjustment_gas = read_cell(ws, 'A4')
            efficiency_adjustment_coal = read_cell(ws, 'A5')

            # Save the workbook
            wb.save()
            wb.close()
        finally:
            app.quit()

    except PermissionError as e:
        print(f"PermissionError: {e}")
        print(f"Could not overwrite {output_path}. Please close the file if it is open in Excel and try again.")
        raise
    except Exception as e:
        print(f"Error copying template: {e}")
        # As a fallback, create a new workbook from the template using xlwings
        try:
            app = xw.App(visible=False)
            try:
                wb_template = app.books.open(template_path)
                wb_template.save(output_path)
                wb_template.close()
                print(f"Created {output_path} from template using xlwings.")
            finally:
                app.quit()
        except Exception as e2:
            print(f"Failed to create workbook from template: {e2}")
            raise


"""
For a given ISO code, compute utilization factors from IRENA and EMBER,
and export both tables to a single Excel file (two sheets, one for each source).
"""

# --- IRENA Utilization Factors ---

# When filtering for iso, groupby fields iso_code, model_fuel, and year
df_irena_c_iso = (
    df_irena_c[df_irena_c['iso_code'] == input_iso]
    .groupby(['iso_code', 'model_fuel', 'Year'], as_index=False)
    .agg({'Electricity statistics (MW/GWh)': lambda x: x.sum() / 1000})
)
df_irena_g_iso = (
    df_irena_g[df_irena_g['iso_code'] == input_iso]
    .groupby(['iso_code', 'model_fuel', 'Year'], as_index=False)
    .agg({'Electricity statistics (MW/GWh)': lambda x: x.sum() / 1000})
)

# Standardize column names for merging
df_irena_c_iso = df_irena_c_iso.rename(columns={
    'Electricity statistics (MW/GWh)': 'Capacity_GW'
})
df_irena_g_iso = df_irena_g_iso.rename(columns={
    'Electricity statistics (MW/GWh)': 'Generation_TWh'
})

# Merge on Country, Type, Year
df_irena_util = pd.merge(
    df_irena_c_iso[['iso_code', 'model_fuel', 'Year', 'Capacity_GW']],
    df_irena_g_iso[['iso_code', 'model_fuel', 'Year', 'Generation_TWh']],
    on=['iso_code', 'model_fuel', 'Year'],
    how='inner'
)

# Compute utilization factor: (Generation in GWh) / (Capacity in MW * 8.76)
df_irena_util['utilization_factor'] = df_irena_util['Generation_TWh'] / (df_irena_util['Capacity_GW'] * 8.76)

# Pivot table: Type x Year, values = Utilization_Factor
util_pivot_irena = df_irena_util.pivot_table(
    index=['iso_code', 'model_fuel'],
    columns='Year',
    values='utilization_factor'
)

# --- EMBER Utilization Factors ---

# Get capacity (GW) by country code, year, Type
# When creating cap and gen dfs, sum Value on the 3 indexes (iso_code, Year, model_fuel)
df_capacity = df_ember[
    (df_ember['Unit'] == 'GW') & 
    (df_ember['iso_code'] == input_iso)
].copy()
df_capacity = (
    df_capacity
    .groupby(['iso_code', 'Year', 'model_fuel'], as_index=False)['Value']
    .sum()
)
df_capacity = df_capacity.rename(columns={'Value': 'Capacity_GW'})

df_generation = df_ember[
    (df_ember['Unit'] == 'TWh') & 
    (df_ember['iso_code'] == input_iso)
].copy()
df_generation = (
    df_generation
    .groupby(['iso_code', 'model_fuel', 'Year'], as_index=False)['Value']
    .sum()
)
df_generation = df_generation.rename(columns={'Value': 'Generation_TWh'})
# Merge on Country code, Year, Type
df_ember_util = pd.merge(
    df_capacity,
    df_generation[['iso_code', 'model_fuel', 'Year', 'Generation_TWh']],
    on=['iso_code', 'model_fuel', 'Year'],
    how='inner'
)

# Compute utilization factor: (Generation in GWh) / (Capacity in GW * 8760)
df_ember_util['utilization_factor'] = df_ember_util['Generation_TWh'] / (df_ember_util['Capacity_GW'] * 8.76)
# Pivot table for easier viewing
util_pivot_ember = df_ember_util.pivot_table(
    index=['iso_code', 'model_fuel'],
    columns='Year',
    values='utilization_factor'
)


# existing stock characteristics
#############################################################################################################################
costs_df = pd.read_excel("data/technologies/ep_technoeconomic_assumptions.xlsx", sheet_name="costs")
costs_size_multipliers_df = pd.read_excel("data/technologies/ep_technoeconomic_assumptions.xlsx", sheet_name="costs_size_multipliers")
reg_mult_df = pd.read_excel("data/technologies/ep_technoeconomic_assumptions.xlsx", sheet_name="regional_multipliers")
reg_map_df = pd.read_excel("data/technologies/ep_technoeconomic_assumptions.xlsx", sheet_name="ep_regionmap")
thermal_eff_df = pd.read_excel("data/technologies/ep_technoeconomic_assumptions.xlsx", sheet_name="thermal_eff")
re_units_cf_grid_cell_mapping = pd.read_csv("data/GlobalEnergyMonitor/re_units_cf_grid_cell_mapping.csv")

# Example: View costs_df for a particular model_name using DuckDB SQL

# Register the DataFrame as a DuckDB table
duckdb.register('costs_df', costs_df)
duckdb.register('costs_size_multipliers_df', costs_size_multipliers_df)
duckdb.register('reg_mult_df', reg_mult_df)
duckdb.register('reg_map_df', reg_map_df)
duckdb.register('thermal_eff_df', thermal_eff_df)


def get_costs_and_eff(input_iso, input_size, input_model_name, input_year):
    # Get all year columns from thermal_eff_df that are less than input_year
    year_cols = [col for col in thermal_eff_df.columns if str(col).isdigit() and int(col) < input_year]
    if year_cols:
        max_year = max(year_cols, key=lambda x: int(x))
    else:
        max_year = min([col for col in thermal_eff_df.columns if str(col).isdigit()], key=lambda x: int(x))

    # Check if model_name exists in thermal_eff_df
    eff_rows = thermal_eff_df[thermal_eff_df['model_name'].str.lower() == input_model_name.lower()]
    if eff_rows.empty:
        # No efficiency data for this model_name: skip the efficiency join, set efficiency=1
        result = duckdb.query(f"""
            SELECT 
                T1.capex * T2.capex * T3.capex as capex, 
                T1.fixom * T2.fixom * T3.fixom as fixom, 
                T1.varom * T2.varom * T3.varom as varom, 
                1 as efficiency
            FROM
            (SELECT * from costs_df WHERE lower(model_name) = lower('{input_model_name}')) T1
            CROSS JOIN
            (SELECT * from costs_size_multipliers_df WHERE size = (SELECT max(size) from costs_size_multipliers_df where size < {input_size})) T2
            CROSS JOIN
            (Select T1.* from reg_mult_df T1 INNER JOIN reg_map_df T2 ON T1.region = T2.region WHERE T2.iso = '{input_iso}') T3
        """).to_df()
    else:
        # Query the table for the particular model_name, including efficiency
        result = duckdb.query(f"""
            SELECT 
                T1.capex * T2.capex * T3.capex as capex, 
                T1.fixom * T2.fixom * T3.fixom as fixom, 
                T1.varom * T2.varom * T3.varom as varom, 
                T4.efficiency * T3.efficiency as efficiency
            FROM
            (SELECT * from costs_df WHERE lower(model_name) = lower('{input_model_name}')) T1
            CROSS JOIN
            (SELECT * from costs_size_multipliers_df WHERE size = (SELECT max(size) from costs_size_multipliers_df where size < {input_size})) T2
            CROSS JOIN
            (Select T1.* from reg_mult_df T1 INNER JOIN reg_map_df T2 ON T1.region = T2.region WHERE T2.iso = '{input_iso}') T3
            CROSS JOIN
            (SELECT "{max_year}" as efficiency from thermal_eff_df where lower(model_name) = lower('{input_model_name}') AND size = (SELECT max(size) from thermal_eff_df where size < {input_size})) T4
        """).to_df()

    return result


# Filter df_gem for a particular iso, e.g., 'USA'
df_gem_iso = df_gem[df_gem['iso_code'] == input_iso]


# Keep only rows from df_gem_iso where 'Status' is in a certain list of values
statuses_to_keep = ['operating', 'construction', 'mothballed']  # example statuses to keep
df_gem_iso = df_gem_iso[df_gem_iso['Status'].isin(statuses_to_keep)]

df_gem_iso.to_excel("output/df_gem_iso_before_gap_filling.xlsx", index=False)

def is_valid_year(val):
    try:
        year = int(val)
        # Consider valid if year is between 1900 and 2100 (adjust as needed)
        return 1900 <= year <= 2100
    except (ValueError, TypeError):
        return False

def get_start_year(row):
    status = str(row['Status']).lower()
    year_val = row['Start year']
    if not is_valid_year(year_val):
        if status == 'construction':
            return 2028
        else:
            return 2015
    return int(year_val)

df_gem_iso['Start year'] = df_gem_iso.apply(get_start_year, axis=1)

# Check for missing renewable capacity from IRENA and add to df_gem_iso if needed
def add_missing_irena_capacity(df_gem_iso, df_irena_c, input_iso, fuel_type):
    """
    Compare IRENA 2022 renewable capacity with cumulative GEM capacity (years ≤ 2022).
    If IRENA > GEM, add the difference as a new record for 2022.
    """
    
    # Technology mapping for different fuel types
    tech_mapping = {
        'solar': {'Technology': 'PV', 'Type': 'solar', 'description': 'Solar'},
        'wind': {'Technology': 'Onshore', 'Type': 'wind', 'description': 'Wind'},
        'hydro': {'Technology': 'conventional storage', 'Type': 'hydropower', 'description': 'Hydro'}
    }
    
    if fuel_type not in tech_mapping:
        print(f"Warning: Unknown fuel type '{fuel_type}'. Skipping.")
        return df_gem_iso
    
    # Get IRENA capacity for 2022
    irena_capacity_2022 = df_irena_c[
        (df_irena_c['iso_code'] == input_iso) & 
        (df_irena_c['model_fuel'] == fuel_type) & 
        (df_irena_c['Year'] == 2022)
    ]['Electricity statistics (MW/GWh)'].sum() / 1000  # Convert MW to GW
    
    # Get cumulative GEM capacity for years <= 2022
    gem_capacity_cumulative = df_gem_iso[
        (df_gem_iso['model_fuel'] == fuel_type) & 
        (df_gem_iso['Start year'] <= 2022)
    ]['Capacity (MW)'].sum() / 1000  # Convert MW to GW
    
    # Calculate the difference
    capacity_difference_gw = irena_capacity_2022 - gem_capacity_cumulative
    
    print(f"IRENA {fuel_type} capacity 2022: {irena_capacity_2022:.2f} GW")
    print(f"GEM cumulative {fuel_type} capacity (≤2022): {gem_capacity_cumulative:.2f} GW")
    print(f"Difference: {capacity_difference_gw:.2f} GW")
    
    # If difference > 0, add missing capacity
    if capacity_difference_gw > 0:
        print(f"Adding {capacity_difference_gw:.2f} GW of missing {fuel_type} capacity to df_gem_iso")
        
        # Create new row with missing capacity for year 2022
        tech_info = tech_mapping[fuel_type]
        new_row = pd.Series({
            'iso_code': input_iso,
            'model_fuel': fuel_type,
            'Capacity (MW)': capacity_difference_gw * 1000,  # Convert back to MW
            'Plant / Project name': 'Aggregated Plant - IRENA Gap',
            'Unit / Phase name': f'Missing {tech_info["description"]} Capacity',
            'GEM unit/phase ID': None,
            'Status': 'operating',
            'Start year': 2022,
            'Technology': tech_info['Technology'],
            'Type': tech_info['Type']
        })
        
        # Add the new row to df_gem_iso
        df_gem_iso = pd.concat([df_gem_iso, new_row.to_frame().T], ignore_index=True)
        print(f"Added new {fuel_type} record for year 2022 with {capacity_difference_gw:.2f} GW")
    else:
        print(f"No missing {fuel_type} capacity to add")
    
    return df_gem_iso

# Apply the function to add missing IRENA capacity for solar and wind
df_gem_iso = add_missing_irena_capacity(df_gem_iso, df_irena_c, input_iso, 'solar')
df_gem_iso = add_missing_irena_capacity(df_gem_iso, df_irena_c, input_iso, 'wind')
df_gem_iso = add_missing_irena_capacity(df_gem_iso, df_irena_c, input_iso, 'hydro')

# Check for missing thermal capacity from EMBER and add to df_gem_iso if needed
def add_missing_ember_capacity(df_gem_iso, df_ember, input_iso, fuel_type):
    """
    Compare EMBER 2022 thermal capacity with cumulative GEM capacity (years ≤ 2022).
    If EMBER > GEM, add the difference as a new record for 2022.
    """
    
    # Technology mapping for different fuel types
    tech_mapping = {
        'bioenergy': {'Technology': 'bioenergy', 'Type': 'bioenergy', 'description': 'Bioenergy'},
        'coal': {'Technology': 'subcritical', 'Type': 'coal', 'description': 'Coal'},
        'gas': {'Technology': 'combined cycle', 'Type': 'gas', 'description': 'Gas'},
        'oil': {'Technology': 'gas turbine', 'Type': 'oil', 'description': 'Oil'}
    }
    
    if fuel_type not in tech_mapping:
        print(f"Warning: Unknown fuel type '{fuel_type}'. Skipping.")
        return df_gem_iso
    
    # Get EMBER capacity for 2022
    ember_capacity_2022 = df_ember[
        (df_ember['iso_code'] == input_iso) & 
        (df_ember['model_fuel'] == fuel_type) & 
        (df_ember['Year'] == 2022) &
        (df_ember['Unit'] == 'GW')
    ]['Value'].sum()  # Already in GW
    
    # Get cumulative GEM capacity for years <= 2022
    gem_capacity_cumulative = df_gem_iso[
        (df_gem_iso['model_fuel'] == fuel_type) & 
        (df_gem_iso['Start year'] <= 2022)
    ]['Capacity (MW)'].sum() / 1000  # Convert MW to GW
    
    # Calculate the difference
    capacity_difference_gw = ember_capacity_2022 - gem_capacity_cumulative
    
    print(f"EMBER {fuel_type} capacity 2022: {ember_capacity_2022:.2f} GW")
    print(f"GEM cumulative {fuel_type} capacity (≤2022): {gem_capacity_cumulative:.2f} GW")
    print(f"Difference: {capacity_difference_gw:.2f} GW")
    
    # If difference > 0, add missing capacity
    if capacity_difference_gw > 0:
        print(f"Adding {capacity_difference_gw:.2f} GW of missing {fuel_type} capacity to df_gem_iso")
        
        # Create new row with missing capacity for year 2022
        tech_info = tech_mapping[fuel_type]
        new_row = pd.Series({
            'iso_code': input_iso,
            'model_fuel': fuel_type,
            'Capacity (MW)': capacity_difference_gw * 1000,  # Convert back to MW
            'Plant / Project name': 'Aggregated Plant - EMBER Gap',
            'Unit / Phase name': f'Missing {tech_info["description"]} Capacity',
            'GEM unit/phase ID': None,
            'Status': 'operating',
            'Start year': 2022,
            'Technology': tech_info['Technology'],
            'Type': tech_info['Type']
        })
        
        # Add the new row to df_gem_iso
        df_gem_iso = pd.concat([df_gem_iso, new_row.to_frame().T], ignore_index=True)
        print(f"Added new {fuel_type} record for year 2022 with {capacity_difference_gw:.2f} GW")
    else:
        print(f"No missing {fuel_type} capacity to add")
    
    return df_gem_iso

# Apply the function to add missing EMBER capacity for bioenergy, coal, and gas
df_gem_iso = add_missing_ember_capacity(df_gem_iso, df_ember, input_iso, 'bioenergy')
df_gem_iso = add_missing_ember_capacity(df_gem_iso, df_ember, input_iso, 'coal')
df_gem_iso = add_missing_ember_capacity(df_gem_iso, df_ember, input_iso, 'gas')


# Join df_gem_iso with df_gem_map on 'model_fuel' and 'Technology'
# Using left join to preserve gap-filling records that might not have exact mappings
df_gem_iso = df_gem_iso.merge(df_gem_map[['model_fuel', 'Technology', 'model_name']], left_on=['model_fuel', 'Technology'], right_on=['model_fuel', 'Technology'], how='left')

# Fill missing model_names for gap-filling records
df_gem_iso['model_name'] = df_gem_iso['model_name'].fillna(df_gem_iso['model_fuel'].apply(lambda x: f'ep_{x}'))

# add capacity factor and grid cell to df_gem_iso
df_gem_iso = df_gem_iso.merge(re_units_cf_grid_cell_mapping,
    left_on=['GEM unit/phase ID'], 
    right_on=['GEM_unit/phase_ID'], 
    how='left')


df_gem_iso.to_excel("output/df_gem_iso_after_gap_filling.xlsx", index=False)


# df_gem_iso.to_excel("output/df_gem_iso.xlsx", index=False)


def apply_get_costs_and_eff(row):
    iso = row['iso_code']
    size = row['Capacity (MW)'] if 'Capacity (MW)' in row else None
    model_name = row['model_name'].lower() if 'model_name' in row else None
    year = row['Start year'] if 'Start year' in row else None
    if None in (iso, size, model_name, year):
        return [None, None, None, None]
    try:
        res = get_costs_and_eff(iso, size, model_name, year)
        return res.iloc[0].tolist()
    except Exception as e:
        return [None, None, None, None]

# Define the names of the new columns
new_cols = ['capex', 'fixom', 'varom', 'efficiency']

df_gem_iso[new_cols] = df_gem_iso.apply(apply_get_costs_and_eff, axis=1, result_type='expand')

# Interactive DuckDB query to find records with null Start year
# query = duckdb.sql("SELECT * FROM df_gem_iso WHERE \"Start year\" IS NULL").df()
# display(query)

# Calibrate efficiency for gas and coal to match UNSD fuel consumption
df_gem_iso.loc[df_gem_iso['model_name'].str.lower().str.startswith('ep_gas'), 'efficiency'] = (
    df_gem_iso.loc[df_gem_iso['model_name'].str.lower().str.startswith('ep_gas'), 'efficiency'] * efficiency_adjustment_gas
)

df_gem_iso.loc[df_gem_iso['model_name'].str.lower().str.startswith('ep_coal'), 'efficiency'] = (
    df_gem_iso.loc[df_gem_iso['model_name'].str.lower().str.startswith('ep_coal'), 'efficiency'] * efficiency_adjustment_coal
)


#############################################################################################################################
# need to look into why ep_solar_PV is turning up with no efficiency. forcing an update here
df_gem_iso.loc[(df_gem_iso['efficiency'].isnull()) | (df_gem_iso['efficiency'] == 0), 'efficiency'] = 0.33123
#############################################################################################################################

query = f"""
    SELECT  
        (
            CASE 
                WHEN "Capacity (MW)" >= {capacity_threshold} THEN CAST(model_name AS VARCHAR) || '_' || COALESCE(CAST("GEM unit/phase ID" AS VARCHAR), '')
                ELSE CAST(model_name AS VARCHAR)
            END
            ||
            CASE 
                WHEN lower(Status) NOT IN ('operating', 'construction') THEN '__m'
                ELSE ''
            END
        ) AS model_name,
        CASE
            WHEN "Capacity (MW)" >= {capacity_threshold} THEN CAST("Plant / Project name" AS VARCHAR) || '_' || COALESCE(CAST("Unit / Phase name" AS VARCHAR), '')
            ELSE 'Aggregated Plant'
        END AS model_description,
        model_fuel,
        iso_code, 
        "Start year",
        CASE 
            WHEN "Capacity (MW)" >= {capacity_threshold} THEN coalesce(CAST("Retired year" AS VARCHAR), '')
            ELSE ''
        END AS "retirement_year",
        CASE 
            WHEN "Capacity (MW)" >= {capacity_threshold} THEN coalesce(CAST("Subnational unit (state, province)" AS VARCHAR), '')
            ELSE ''
        END AS "state",
        CASE 
            WHEN "Capacity (MW)" >= {capacity_threshold} THEN coalesce(CAST("City" AS VARCHAR), '')
            ELSE ''
        END AS "city",        
        CASE 
            WHEN lower(Status) IN ('operating', 'construction') THEN 'active'
            ELSE 'mothballed'
        END AS status_group,
        SUM("Capacity (MW)") / 1000 AS Capacity_GW,
        SUM("Capacity (MW)" * capex) / NULLIF(SUM("Capacity (MW)"), 0) AS capex,
        SUM("Capacity (MW)" * fixom) / NULLIF(SUM("Capacity (MW)"), 0) AS fixom,
        SUM("Capacity (MW)" * varom) / NULLIF(SUM("Capacity (MW)"), 0) AS varom,
        SUM("Capacity (MW)" * efficiency) / NULLIF(SUM("Capacity (MW)"), 0) AS efficiency
    FROM df_gem_iso
    GROUP BY iso_code, "Start year", model_fuel,
        CASE 
            WHEN "Capacity (MW)" >= {capacity_threshold} THEN coalesce(CAST("Retired year" AS VARCHAR), '')
            ELSE ''
        END,
        CASE 
            WHEN lower(Status) IN ('operating', 'construction') THEN 'active'
            ELSE 'mothballed'
        END,
        (
            CASE 
                WHEN "Capacity (MW)" >= {capacity_threshold} THEN CAST(model_name AS VARCHAR) || '_' || COALESCE(CAST("GEM unit/phase ID" AS VARCHAR), '')
                ELSE CAST(model_name AS VARCHAR)
            END
            ||
            CASE 
                WHEN lower(Status) NOT IN ('operating', 'construction') THEN '__m'
                ELSE ''
            END
        ),
        CASE
            WHEN "Capacity (MW)" >= {capacity_threshold} THEN CAST("Plant / Project name" AS VARCHAR) || '_' || COALESCE(CAST("Unit / Phase name" AS VARCHAR), '')
            ELSE 'Aggregated Plant'
        END,
        CASE 
            WHEN "Capacity (MW)" >= {capacity_threshold} THEN coalesce(CAST("Subnational unit (state, province)" AS VARCHAR), '')
            ELSE ''
        END,
        CASE 
            WHEN "Capacity (MW)" >= {capacity_threshold} THEN coalesce(CAST("City" AS VARCHAR), '')
            ELSE ''
        END
"""

df_grouped_gem = duckdb.sql(query).df()

# add UC parameters to df_grouped_gem

def load_uc_data():
    """Load unit commitment data and mappings"""
    uc_data = pd.read_excel('data/technologies/advanced_parameters.xlsx', sheet_name='uc_data')
    uc_tech_map = pd.read_excel('data/technologies/advanced_parameters.xlsx', sheet_name='uc_tech_map')
    return uc_data, uc_tech_map

def find_technology_for_model(model_name, uc_tech_map):
    """Find technology using simple startswith matching"""
    for _, row in uc_tech_map.iterrows():
        technology = row['technology']
        model_patterns = row['model_name'].split(',')
        
        for pattern in model_patterns:
            pattern = pattern.strip()
            if model_name.startswith(pattern):
                return technology
    
    return None

def determine_size_class(technology, capacity_mw):
    """Determine size class based on technology and capacity in MW"""
    size_rules = {
        'OCGT (Peaker)': [(50, '<50 MW'), (200, '50-200 MW'), (float('inf'), '>200 MW')],
        'CCGT': [(300, '<300 MW'), (float('inf'), '>300 MW')],
        'Gas/Oil Steam': [(200, '<200 MW'), (float('inf'), '>200 MW')],
        'Diesel': [(20, '<20 MW'), (float('inf'), '>20 MW')],
        'Subcritical Coal': [(300, '<300 MW'), (float('inf'), '>300 MW')],
        'Supercritical Coal': [(500, '<500 MW'), (float('inf'), '>500 MW')],
        'Nuclear': [(float('inf'), 'All')]
    }
    
    if technology in size_rules:
        for threshold, size_class in size_rules[technology]:
            if capacity_mw < threshold:
                return size_class
    return None

def get_uc_parameters(uc_data, technology, size_class):
    """Get unit commitment parameters for given technology and size class"""
    mask = (uc_data['technology'] == technology) & (uc_data['Size Class'] == size_class)
    matching_rows = uc_data[mask]
    
    if len(matching_rows) == 0:
        return None
    
    row = matching_rows.iloc[0]
    return {
        'min_stable_factor_pct': row['Min Stable Factor (%)'],
        'min_up_time_h': row['Min Up Time (h)'],
        'min_down_time_h': row['Min Down Time (h)'],
        'max_ramp_up_pct_h': row['Max Ramp Up (%/h)'],
        'max_ramp_down_pct_h': row['Max Ramp Down (%/h)'],
        'startup_time_h': row['Startup Time (h)'],
        'startup_cost_per_mw': row['Startup Cost ($/MW)'],
        'shutdown_cost_per_mw': row['Shutdown Cost ($/MW)']
    }

def add_uc_parameters_to_df(df, uc_data, uc_tech_map):
    """Add unit commitment parameters using simple startswith matching"""
    df = df.copy()
    
    # Initialize new columns
    df['uc_technology'] = ''
    df['uc_size_class'] = ''
    
    for col in ['min_stable_factor_pct', 'min_up_time_h', 'min_down_time_h', 
                'max_ramp_up_pct_h', 'max_ramp_down_pct_h', 'startup_time_h', 
                'startup_cost_per_mw', 'shutdown_cost_per_mw']:
        df[col] = np.nan
    
    # Track stats
    matched_count = 0
    total_thermal = 0
    
    # Process each plant
    for idx, row in df.iterrows():
        model_name = row['model_name']
        capacity_mw = row['Capacity_GW'] * 1000  # Convert GW to MW
        
        # Simple startswith matching - much cleaner!
        technology = find_technology_for_model(model_name, uc_tech_map)
        
        if technology is None:
            continue  # Skip non-thermal plants
        
        total_thermal += 1
        size_class = determine_size_class(technology, capacity_mw)
        
        if size_class:
            uc_params = get_uc_parameters(uc_data, technology, size_class)
            if uc_params:
                df.at[idx, 'uc_technology'] = technology
                df.at[idx, 'uc_size_class'] = size_class
                
                for param_name, param_value in uc_params.items():
                    df.at[idx, param_name] = param_value
                
                matched_count += 1
    
    # print(f"🎯 UC Results: {matched_count}/{total_thermal} thermal plants matched ({matched_count/total_thermal*100:.1f}%)")
    return df

# Apply to your Japanese dataframe:
uc_data, uc_tech_map = load_uc_data()
df_grouped_gem_with_uc = add_uc_parameters_to_df(df_grouped_gem, uc_data, uc_tech_map)

df_grouped_gem_with_uc = df_grouped_gem_with_uc.drop(columns=['uc_technology', 'uc_size_class'])


output_path = f"output/VerveStacks_{input_iso}.xlsx"

if os.path.exists(output_path):
    # Using xlwings to write existing stock data
    app = xw.App(visible=False)
    try:
        wb = app.books.open(output_path)
        
        # Create or replace the sheet
        if 'existing_stock' in [ws.name for ws in wb.sheets]:
            wb.sheets['existing_stock'].delete()
        
        ws = wb.sheets.add('existing_stock')
        
        # Write the data
        df_sorted = df_grouped_gem_with_uc.sort_values(by=['model_name', 'Start year'])
        ws.range('A1').value = [df_sorted.columns.tolist()] + df_sorted.values.tolist()
        
        wb.save()
        wb.close()
    finally:
        app.quit()


# function calls
#############################################################################################################################


calibration_data(input_iso,df_irena_util,df_ember_util,df_grouped_gem)
ccs_retrofits(input_iso,df_grouped_gem)
re_targets_ember(input_iso)
get_weo_data(input_iso)
get_iamc_data(input_iso)
